<a href="https://colab.research.google.com/github/chenyaofo/statistical-learning-method-numpy-impl/blob/main/algorithms/naive_bayes_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
import numpy
import requests
from io import BytesIO
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split

In [122]:
# we can choose a specific datasets from libsvm (https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/)
# here we choose dataset australian (https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html#australian)
# set the download_link from the website (note that here we download the scaled version)
download_link = "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/australian_scale"
# set the #features according to the info in the website
n_features = 14
# set the test set proportion to 50%
test_size = 0.5

In [123]:
def download_process_data(download_link, n_features, test_size):
    # download the dataset in a online way
    r = requests.get(download_link)

    # load the dataset by sklearn api
    X, y = load_svmlight_file(BytesIO(r.content), n_features=n_features)
    X = X.toarray()

    n_samples, n_features = X.shape
    X = numpy.column_stack((X, numpy.ones((n_samples, 1))))
    y = y.reshape((-1, 1))

    # the default type of label is float, we cast it into int32
    y = y.astype(numpy.int32)

    # we transform label -1 to 0 since the function 'bincount' only receives non-negative inputs
    y[y==-1]=0

    print(max(y))
    print(min(y))

    if test_size is not None:
        # split the dataset into train and val sets by sklearn api
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5)
        return X_train, X_val, y_train, y_val
    else:
        return X, y

In [124]:
# the shape of X_train is (#train_set, n_features)
# the shape of X_val is (#val_set, n_features)
# the shape of y_train is (#train_set, 1)
# the shape of y_val is (#val_set, 1)
# in this dataset, each sample has 14-dimension features, each dimension is a real number, the label belong to {0,1}
X_train, X_val, y_train, y_val = download_process_data(download_link, n_features, test_size)

# we find the maximum label in train and val sets, which is a prameter for the function 'bincount'
maximum_label = numpy.max(numpy.concatenate([y_train, y_val], axis=0))

X_train = X_train[:, :13]
X_val = X_val[:, :13]
n_features = 13

[1]
[0]


In [125]:
# P(label | features) = P(features | label) * P(label) / P(features)

In [126]:
# step 1: calculate P(features | label)

# sicne P(features) is a joint distibution, we calculate it based on the assumption that
# each feature is independent and identically distributed
# here we assume each feature follows a Gaussian distribution
# thus we calcuate the mean and std of Gaussian distribution

def gaussian_distribution_pdf(x, mean, std):
    return (1 / (numpy.sqrt(2 * numpy.pi) * std+1e-8)) * numpy.exp(-(x-mean)**2/(2*std**2+1e-8))
# y_train[-1,-1]=2
a = numpy.hstack((X_train,y_train))
a = a[a[:, -1].argsort()]
a = numpy.split(a[:,:-1], numpy.unique(a[:, -1], return_index=True)[1][1:])
# print(a[2].shape)
for g in a:
    print(g.shape)
print(max(y_train))
print(min(y_train))
means = []
stds = []

for group in a:
    mean = numpy.mean(group, axis=0, keepdims=True)
    std = numpy.std(group, axis=0, keepdims=True)
    means.append(mean)
    stds.append(std)

means = numpy.concatenate(means, axis=0) # (max_label, n_features)
stds = numpy.concatenate(stds, axis=0) # (max_label, n_features)

print(means.shape)

p_features_label = gaussian_distribution_pdf(X_val, numpy.expand_dims(means, 1), numpy.expand_dims(stds, 1)) #(#val_set, n_features) (max_label, n_features) (max_label, n_features)
print(p_features_label.shape) # (max_label, #val_set, n_features)

(180, 13)
(165, 13)
[1]
[0]
(2, 13)
(2, 345, 13)


In [127]:
# step 2: calculate P(label)

# the marginal distribution of label y can be directly calculated:
label_freq = numpy.bincount(y_train.reshape(-1), minlength=maximum_label+1)
p_label = label_freq / label_freq.sum() # (max_label,)

In [128]:
print(p_label)

[0.52173913 0.47826087]


In [129]:
# step 3: calculate P(features)
global_mean = numpy.mean(X_train, axis=0, keepdims=True)
global_std = numpy.std(X_train, axis=0, keepdims=True)

p_features = gaussian_distribution_pdf(X_val, global_mean, global_std)
print(p_features.shape) # (#val_set, n_features)

(345, 13)


In [130]:
def compute_accuracy(y_gt:numpy.array, y_predict:numpy.array):
    # number of correct samples / number of total samples
    return (y_gt == y_predict).sum() / y_gt.size

In [131]:
def classify_baive_bayes():
    pass

predict_score = numpy.log(p_features_label).sum(axis=-1) * numpy.expand_dims(p_label,axis=1) / numpy.log(p_features).sum(axis=-1)

In [132]:
print(predict_score.shape)

(2, 345)


In [133]:
y_predict = predict_score.argmin(0)

In [134]:
print(y_predict.shape)

(345,)


In [135]:
y_predict = numpy.expand_dims(y_predict,axis=1)

In [136]:
# compute the accuracy on predicted samples of the val set
print(f"The accuracy on val set is {compute_accuracy(y_val, y_predict)*100.0:.2f}%")

The accuracy on val set is 87.25%
